### Quiz Builder

In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

True

In [3]:
# Structured output for model for quiz output

from typing import List


class QuestionOptions(BaseModel):
    A: str = Field("Option A for the question")
    B: str = Field("Option B for the question")
    C: str = Field("Option C for the question")
    D: str = Field("Option D for the question")


class Question(BaseModel):
    question: str = Field(
        description="Question")
    options: QuestionOptions
    correctAnswer: str = Field(description="Correct answer - one of these options A, B, C or D")
    hint: str = Field(
        description="A small hint to help user think in right direction")
    explaination: str = Field(description="Explaination of the correct answer")
    

class QuizOutput(BaseModel):
    title: str = Field(description="A suitable title for the quiz.")
    questions: List[Question] = Field(
        description="A list of quiz questions in MCQ format, with each question having 4 options."
    )

In [5]:
# output structures
class SummaryNoteOutput(BaseModel):
    title: str = Field(description="Title of the note")
    summary: str = Field(description="Summary Note in markdown language")

In [7]:
# Student profile type definition
from typing import Optional, TypedDict


class StudentProfile(TypedDict):
    gender: str
    class_level: str  # e.g., "class 6", "12th", "undergrad", "postgrad"
    language: str     # e.g., "hindi", "english", "marathi"


class UserPrompt(TypedDict):
    topic: str
    file_url: Optional[str]
# Graph state


class State(TypedDict):
    learning_space_id: str
    student_profile: StudentProfile
    user_prompt: UserPrompt
    summary_notes: SummaryNoteOutput
    podcast_script: str
    mindmap: str
    quiz: QuizOutput
    recommended_resources: str
    study_plan: str
    combined_output: str

In [4]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai", )
response = model.invoke("Hello World!!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--ea6e2e2e-a19a-41ea-9855-f4c1e3bcd065-0' usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}


In [8]:
# Agent Node - Notes Summary

def node_quiz(state: State):
    """LLM call to generate quiz based on summary content"""

    # generate the chat prompt

    """Create a personalized prompt based on student profile"""

    prompt_template = ChatPromptTemplate([
        ("system", """ 
        You are a helpful academic tutor. Use these instructions to create a quiz on the notes provided by the user:
        
        Student Profile:
            - Class Level: {grade_level}
            - Language: {language} 
            - Gender: {gender}
        
        1. Questions should be in MCQ format with 4 options each.
        2. Create 10 quality questions which tests fundamentals and analytical thinking of the user. 
        3. Adapt your language and complexity based on the student's profile provided.
        4. Respond in JSON format which can be used to render a quiz UI.
        5. Include correct answer, hint and explaination with each question.
        """),
        ("user", "Topic Summary {topic_summary}")
    ])
    
   
    # init a new model with structured output
    model = init_chat_model(
        "gemini-2.0-flash", model_provider="google_genai").with_structured_output(QuizOutput)

    chain = prompt_template | model
    
    response = chain.invoke(
        {
            "grade_level": state['student_profile'].get("grade_level", "general"),
            "language": state['student_profile'].get("language", "English"),
            "gender": state['student_profile'].get("gender", ""),
            "topic_summary": state["summary_notes"]
        }
    )

    # # update in supabase database
    # supabase_response = (
    #     supabase.table("learning_space")
    #     .update({"summary_notes": response.model_dump()})
    #     .eq("id", state["learning_space_id"])
    #     .execute()
    # )

    # print("Supabase Response", supabase_response)

    return {"quiz": response.model_dump()}

In [9]:
input_state = {
    "learning_space_id": '48484754-ae7c-4b4b-bdea-12784ed54a2f',
    "student_profile": {
        "gender": "MALE",
        "grade_level": "12th",
        "language": "HINDI"
    },
    "user_prompt": {
        "topic": "Uncertainty in AI",
        "file_url": "https://kclenjtsvdayeghmmmkm.supabase.co/storage/v1/object/public/learning-sources/public/Slides_Module8.pdf",
    },
    "summary_notes": """
    {"title": "AI में अनिश्चितता (Uncertainty in AI) ", "summary": "# AI में अनिश्चितता (Uncertainty in AI) पर नोट्स\n\n## मॉड्यूल का उद्देश्य:\n*   AI सिस्टम्स में ज्ञान और तर्क में अनिश्चितता को मापने और प्रबंधित करने के तरीकों को समझना और लागू करना।\n\n## विषय:\n*   **अनिश्चितता में काम करना (Acting under Uncertainty):**\n    *   एजेंट्स को अनिश्चितता से निपटने की आवश्यकता हो सकती है।\n    *   एक एजेंट को निश्चित रूप से पता नहीं हो सकता कि वह किस स्थिति में है या कार्यों के क्रम के बाद वह कहां समाप्त होगा।\n    *   एजेंट्स को एक विश्वास स्थिति (belief state) का ट्रैक रखकर अनिश्चितता को संभालने के लिए डिज़ाइन किया गया है।\n    *   उदाहरण: एक स्वचालित टैक्सी का लक्ष्य है कि वह एक यात्री को समय पर हवाई अड्डे पर पहुंचाए।\n\n*   **अनिश्चितता का सारांश (Summarizing uncertainty):**\n    *   अनिश्चित तर्क का एक उदाहरण देखते हैं।\n    *   निदान (Diagnosis) - चाहे चिकित्सा, ऑटोमोबाइल की मरम्मत, या कुछ भी हो - लगभग हमेशा अनिश्चितता शामिल होती है।\n    *   प्रस्तावात्मक तर्क (propositional logic) का उपयोग करके दंत निदान (dental diagnosis) के लिए नियम लिखने का प्रयास करते हैं।\n    *   एक साधारण नियम पर विचार करें:\n        *   दांत दर्द (Toothache) → कैविटी (Cavity)\n        *   दांत दर्द (Toothache) → कैविटी (Cavity) या मसूड़ों की समस्या (GumProblem) या फोड़ा (Abscess) ...\n        *   कैविटी (Cavity) → दांत दर्द (Toothache)\n    *   चिकित्सा निदान जैसे डोमेन से निपटने के लिए तर्क का उपयोग करने का प्रयास तीन मुख्य कारणों से विफल रहता है:\n        *   आलस्य (Laziness)\n        *   सैद्धांतिक अज्ञानता (Theoretical ignorance)\n        *   व्यावहारिक अज्ञानता (Practical ignorance)\n    *   एजेंट का ज्ञान सबसे अच्छा संबंधित वाक्यों में विश्वास की डिग्री प्रदान कर सकता है।\n    *   विश्वास की डिग्री से निपटने के लिए हमारा मुख्य उपकरण प्रायिकता सिद्धांत (probability theory) है।\n    *   प्रायिकता हमारी आलस्य और अज्ञानता से आने वाली अनिश्चितता को सारांशित करने का एक तरीका प्रदान करती है।\n    *   एक भ्रमित करने वाला बिंदु यह है कि निदान के समय, वास्तविक दुनिया में कोई अनिश्चितता नहीं है: रोगी को या तो कैविटी है या नहीं।\n\n*   **अनिश्चितता और तर्कसंगत निर्णय (Uncertainty and rational decisions):**\n    *   सफलता दरों (Success Rates) की तुलना।\n    *   विश्वसनीयता (Reliability) का महत्व।\n    *   ट्रेड-ऑफ और असुविधाएं (Trade-offs and Inconveniences)।\n    *   प्राथमिकताएं और परिणाम (Preferences and Outcomes)।\n    *   उपयोगिता सिद्धांत (Utility Theory)।\n    *   उपयोगिता की सापेक्षता (Relativity of utility)।\n        *   एजेंट परिप्रेक्ष्य (agent perspective) के आधार पर उपयोगिता की व्यक्तिपरकता (Subjectivity)।\n        *   उपयोगिता कार्यों में विविध प्राथमिकताओं का समायोजन (Accommodation)।\n    *   प्रायिकता और उपयोगिता सिद्धांतों का एकीकरण (Integration)।\n    *   अधिकतम अपेक्षित उपयोगिता (Maximum Expected Utility (MEU)) का सिद्धांत।\n    *   विभिन्न डोमेन में अनुप्रयोग (Application)।\n\n*   **बुनियादी प्रायिकता संकेतन (Basic Probability Notation):**\n    *   स्पष्टता और परिशुद्धता (Clarity and Precision):\n        *   असंदिग्ध संचार (Unambiguous communication)।\n        *   औपचारिक प्रतिनिधित्व (Formal representation)।\n    *   उन्नत विश्लेषण (Advanced Analysis) के लिए आधार:\n        *   जटिल मॉडल के लिए बिल्डिंग ब्लॉक (Building block)।\n        *   गणना और एल्गोरिदम (calculations and algorithms) को सुविधाजनक बनाना।\n\n*   **प्रायिकताएं किसके बारे में हैं (What probabilities are about):**\n    *   नमूना स्थान (Sample space)।\n    *   प्रायिकता मॉडल (Probability model)।\n    *   घटनाएँ (Events)।\n    *   बिना शर्त और पूर्व प्रायिकता (Unconditional and Prior probability)।\n    *   साक्ष्य (Evidence)।\n    *   सशर्त और उत्तर प्रायिकता (Conditional and Posterior probability)।\n    *   उत्पाद नियम (Product rule)।\n\n*   **प्रायिकता अभिकथन (probability assertions) में प्रस्तावों की भाषा (language):**\n    *   यादृच्छिक चर (Random variables)।\n    *   डोमेन (Domain)।\n    *   प्रायिकता वितरण (Probability distribution)।\n    *   प्रायिकता घनत्व फलन (Probability density function)।\n    *   संयुक्त प्रायिकता वितरण (Joint probability distribution)।\n    *   पूर्ण संयुक्त प्रायिकता वितरण (Full joint probability distribution)।\n\n*   **प्रायिकता स्वयंसिद्ध (Probability axioms) और उनकी तर्कसंगतता (reasonableness):**\n    *   प्रायिकता स्वयंसिद्ध उन मूलभूत सिद्धांतों का निर्माण करते हैं जिन पर प्रायिकता का पूरा सिद्धांत बना है।\n        *   गैर-नकारात्मकता स्वयंसिद्ध (Non-negativity Axiom)।\n        *   सामान्यीकरण स्वयंसिद्ध (Normalization Axiom)।\n    *   प्रायिकता स्वयंसिद्धों की तर्कसंगतता (Reasonableness):\n        *   तार्किक स्थिरता (Logical Consistency)।\n        *   सहज ज्ञान युक्त अपील (Intuitive Appeal)।\n        *   गणितीय आधार (Mathematical Foundation)।\n        *   अनुप्रयोग (Applicability)।\n    *   समावेश-बहिष्करण सिद्धांत (Inclusion-exclusion principle)।\n\n*   **पूर्ण संयुक्त वितरण (Full Joint Distributions) का उपयोग करके अनुमान (Inference):**\n    *   संभाव्य अनुमान (Probabilistic inference) अनिश्चितता के तहत तर्क और निर्णय लेने की प्रक्रिया है, जो प्रायिकता सिद्धांत (probability theory) से सिद्धांतों का उपयोग करके मान्यताओं को अपडेट करती है या भविष्यवाणियां करती है।\n    *   अनुमान के प्रकार (Type of inference):\n        *   सीमांतीकरण (Marginalization)।\n        *   सशर्त प्रायिकता (Conditional Probability)।\n        *   संयुक्त अनुमान (Joint Inference)।\n    *   संभाव्य अनुमान (Probabilistic inference) की प्रक्रिया:\n        *   समस्या को परिभाषित करें (Define)।\n        *   एक मॉडल का निर्माण करें (Construct)।\n        *   अनुमान (Inference)।\n        *   परिणामों का मूल्यांकन और व्याख्या करें (Evaluate and Interpret)।\n\n*   **स्वतंत्रता (Independence):**\n    *   मान लीजिए कि हम दांत दर्द, कैच, कैविटी की दुनिया के लिए पूर्ण संयुक्त वितरण का विस्तार एक चौथे बाइनरी चर, मौसम (Weather) (W) को जोड़कर करते हैं।\n    *   चार बाइनरी चर के साथ, इन चर के 16 संभावित संयोजन हैं।\n    *   तालिका की प्रत्येक पंक्ति चर T, C, H, और W के एक अद्वितीय संयोजन का प्रतिनिधित्व करती है।\n    *   अंतिम कॉलम घटनाओं के उस विशिष्ट संयोजन की प्रायिकता का प्रतिनिधित्व करता है।\n    *   सीमांत प्रायिकताएँ (Marginal Probabilities)।\n        *   T = True (दांत दर्द) की सीमांत प्रायिकता (Marginal probability):\n        *   W = Sunny (मौसम धूप है) की सीमांत प्रायिकता (Marginal probability):\n    *   उदाहरण के लिए, P(toothache, catch, cavity, cloudy) और P(toothache, catch, cavity) कैसे संबंधित हैं?\n    *   इससे, हम अनुमान लगा सकते हैं,\n    *   वास्तव में, हम सामान्य समीकरण लिख सकते हैं\n    *   इस प्रकार, चार चर के लिए 32-तत्व तालिका को एक 8-तत्व तालिका और एक 4-तत्व तालिका से बनाया जा सकता है।\n    *   पूर्ण स्वतंत्रता का उपयोग करके बड़े संयुक्त वितरण को छोटे वितरणों में फैक्टर करने के उदाहरण।\n    *   स्वतंत्रता के प्रकार (Types):\n        *   सीमांत स्वतंत्रता (Marginal independence)।\n            *   अन्य चर पर कंडीशनिंग के बिना, अलग-अलग माने जाने वाले व्यक्तिगत चर की स्वतंत्रता।\n        *   पूर्ण स्वतंत्रता (Absolute independence)।\n            *   चर की स्वतंत्रता तब भी होती है जब किसी अन्य चर पर कंडीशनिंग की जाती है।\n\n*   **बेयस का नियम (Bayes' Rule) और इसका उपयोग (Use):**\n    *   उत्पाद नियम (product rule) को इस प्रकार लिखा जा सकता है:\n    *   इस समीकरण को बेयस का नियम (Bayes' rule) (जिसे बेयस का कानून या बेयस का प्रमेय भी कहा जाता है) के रूप में जाना जाता है। यह सरल समीकरण संभाव्य अनुमान (probabilistic inference) के लिए अधिकांश आधुनिक AI प्रणालियों को रेखांकित करता है।\n    *   बेयस का नियम संभाव्य अनुमान (probabilistic inference) के लिए मौलिक है और कई आधुनिक AI प्रणालियों को रेखांकित करता है।\n    *   उदाहरण स्पैम फ़िल्टरिंग (Example Spam Filtering)।\n        *   P(H|E): साक्ष्य E दिए जाने पर परिकल्पना H की उत्तर प्रायिकता (Posterior probability)।\n        *   P(E|H): यदि H सत्य है तो साक्ष्य E को देखने की संभावना।\n        *   P(H): परिकल्पना H की पूर्व प्रायिकता (Prior probability)।\n        *   P(E): साक्ष्य E को देखने की सीमांत प्रायिकता (Marginal probability)।\n    *   चिकित्सा निदान (Medical Diagnosis)।"}...
    """
}

In [10]:
response = node_quiz(input_state)

In [12]:
response['quiz']

{'title': 'AI में अनिश्चितता पर प्रश्नोत्तरी',
 'questions': [{'question': 'AI में अनिश्चितता से निपटने के दौरान, निम्नलिखित में से कौन सा परिदृश्य सही है?',
   'options': {'A': 'एजेंट को निश्चित रूप से पता नहीं हो सकता कि वह किस स्थिति में है।',
    'B': 'एजेंट हमेशा जानता है कि वह किस स्थिति में है।',
    'C': 'एजेंट को कार्यों के परिणाम का निश्चित ज्ञान होता है।',
    'D': 'एजेंट अनिश्चितता से प्रभावित नहीं होता है।'},
   'correctAnswer': 'A',
   'hint': 'अनिश्चितता का मतलब निश्चित ज्ञान का अभाव है।',
   'explaination': 'अनिश्चितता में, एजेंट को यह निश्चित रूप से नहीं पता होता कि वह किस स्थिति में है या कार्यों के क्रम के बाद वह कहाँ समाप्त होगा।'},
  {'question': 'AI में अनिश्चितता को सारांशित करने के लिए मुख्य उपकरण क्या है?',
   'options': {'A': 'प्रस्तावात्मक तर्क का उपयोग करके',
    'B': 'प्रायिकता सिद्धांत का उपयोग करके',
    'C': 'निश्चितता कारकों का उपयोग करके',
    'D': 'फजी लॉजिक का उपयोग करके'},
   'correctAnswer': 'B',
   'hint': 'प्रायिकता अनिश्चितता को मापने और प्रबंधि